In [2]:
from rdflib.plugins.stores.sparqlstore import SPARQLUpdateStore
from rdflib import Graph, Namespace, URIRef

from xml.dom.minidom import parse
import xml.etree.ElementTree as etree
from Marc_to_Bibframe.Marc.marcWork import MarcWork
from Marc_to_Bibframe.Marc.marcInstance import MarcInstance
from Marc_to_Bibframe.Marc.marcItems import MarcItems
from Marc_to_Bibframe.Work.work import Work
from Marc_to_Bibframe.Instance.instance import Instance

import pysolr
from Solr.solr import create_doc

In [87]:
#JENA
store = SPARQLUpdateStore(update_endpoint='http://localhost:3030/bibframe/update')
query_endpoint = 'http://localhost:3030/bibframe/query'
update_endpoint = 'http://localhost:3030/bibframe/update'
store.open((query_endpoint, update_endpoint))



In [2]:
#SOLR
solr = pysolr.Solr('http://localhost:8983/solr/search/', timeout=10)
solr.ping()

'{\n  "responseHeader":{\n    "zkConnected":null,\n    "status":0,\n    "QTime":3,\n    "params":{\n      "q":"{!lucene}*:*",\n      "distrib":"false",\n      "df":"_text_",\n      "rows":"10",\n      "echoParams":"all",\n      "rid":"localhost-3"}},\n  "status":"OK"}\n'

In [3]:
def Make_Graph(nt, bf, count ):
    G1 = "PREFIX bk: <http://bibliokeia.com/bibframe/"+bf+"""/>
    PREFIX bf: <http://id.loc.gov/ontologies/bibframe/> 
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX bflc: <http://id.loc.gov/ontologies/bflc/>
    PREFIX madsrdf: <http://www.loc.gov/mads/rdf/v1#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

    INSERT DATA {
        GRAPH bk:"""

    G2 = " {"

    G3 = """}
    }"""

    G = G1+str(count)+" { \n"+nt+"}}"

    return G

In [4]:
def Upadate(path_marc, count, shelf):
    
    marc_file = parse(path_marc)
    records = marc_file.getElementsByTagName('record')

    docs = list()
    for record in records:
        
        marcxml = etree.fromstring(record.toxml())
        workMarc = MarcWork(marcxml)
        instanceMarc = MarcInstance(marcxml)
        itemsMarc = MarcItems(marcxml)
        print(records.index(record))
        print(workMarc.Title().get('title'))

        #SOLR
        doc = create_doc(workMarc, instanceMarc, itemsMarc, 'Livro', shelf, count)
        docs.append(doc)

        #JENA
        BFwork = URIRef(f"http://bibliokeia.com/bibframe/work/{count}") 
        BFinstance = URIRef(f"http://bibliokeia.com/bibframe/instance/{count}")
        #Work
        work = Work(count, workMarc, BFwork, BFinstance)
        work.serialize(f'out/E1/P5/works/{count}.ttl', format='turtle')
        nt = work.serialize(format='nt')
        W = Make_Graph(nt, "work", count)
        #store.update(W)
        #Instance
        instance = Instance(count, workMarc, instanceMarc, itemsMarc, BFwork, BFinstance, shelf)
        i = instance.get('instance')
        #i.serialize(f'out/E1/P2/instances/{count}.ttl', format='turtle')
        i_nt = i.serialize(format='nt')
        I = Make_Graph(i_nt, "instance", count)
        #store.update(I)
        #Items
        items = instance.get('items')
        for item, register in items:
            #item.serialize(f'out/E1/P3/items/{register}.ttl', format='turtle')
            item_nt = item.serialize(format='nt')
            Item = Make_Graph(item_nt, "item", register)
            #store.update(Item)

            
        count += 1

path_marc = r'Input/E1/P5/koha.xml'
count = 86
Upadate(path_marc, count, "E1.P5")

Failed to convert Literal lexical form to value. Datatype=http://www.w3.org/2001/XMLSchema#date, Converter=<function parse_date at 0x000002EDBBB2CEE0>
Traceback (most recent call last):
  File "c:\Users\inaci\Desktop\BiblioKeia\venv\lib\site-packages\rdflib\term.py", line 2084, in _castLexicalToPython
    return conv_func(lexical)  # type: ignore[arg-type]
  File "c:\Users\inaci\Desktop\BiblioKeia\venv\lib\site-packages\isodate\isodates.py", line 201, in parse_date
    return date(sign * int(groups['year']),
ValueError: year 0 is out of range


0
Computer programming in quantitative biology
1
Using PC DOS
2
Estruturas de dados
3
Programacion en Pascal
4
CGI programming with Perl
5
Banco de dados orientado a objeto
6
Diseño de sistemas de computacion
7
Tecnologia da informação
8
Programação Linear com aplicações em microcomputadores
9
Windows 3.1 helpdesk
10
Study and compilation of computer languages
11
CLIPPER 5.0
12
Effective use of ans cobol computer programming language
13
Programação estruturada com estudos de casos em Pascal
14
Aprendendo Delphi 3.0
15
Aprendendo Delphi 2.0
16
XML como programar
17
A simulação da capacidade de suporte para populações agrícolas nos trópicos úmidos
18
PHP 4


In [10]:
solr.add(docs, commit=True)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":379}}\n'

In [5]:
path_marc = r'Input/E1/P5/koha.xml'
marc_file = parse(path_marc)
records = marc_file.getElementsByTagName('record')

count = 16
record = records[count]
marcxml = etree.fromstring(record.toxml())
workMarc = MarcWork(marcxml)
instanceMarc = MarcInstance(marcxml)
itemsMarc = MarcItems(marcxml)
BFwork = URIRef(f"http://bibliokeia.com/bibframe/work/{count}") 
BFinstance = URIRef(f"http://bibliokeia.com/bibframe/instance/{count}")
work = Work(count, workMarc, BFwork, BFinstance)
instance = Instance(count, workMarc, instanceMarc, itemsMarc, BFwork, BFinstance, 'E1.P3')

http://id.loc.gov/vocabulary/languages/ po does not look like a valid URI, trying to serialize this will break.


In [6]:
workMarc.Language()

' po'

In [7]:
workMarc.tag008

'20100452003     rsba           01 1 por '

In [16]:
x = '98-0278 '
x.rstrip()

'98-0278'

In [4]:
g = Graph()
g.parse('cli/Out/works/14649397.bibframe.rdf')

<Graph identifier=Ne84daf5ad74a41efbade85675acd9138 (<class 'rdflib.graph.Graph'>)>